<a href="https://colab.research.google.com/github/nilesh0109/CV2_SoSe_19/blob/master/Saliency_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/CV2 exercies/Archive.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
import numpy as np
import os
import tensorflow as tf
import imageio
from __future__ import division
from skimage.transform import resize

In [0]:
ROOT= '/tmp'
NUM_IMAGES = 1200
def load_train_data():
	training_img_directory = ROOT+'/data/train/images'
	training_fixation_directory = ROOT+'/data/train/fixations'

	train_imgs = np.zeros((NUM_IMAGES, 180, 320, 3), dtype=np.uint8)
	train_fixations = np.zeros((NUM_IMAGES, 180, 320, 1), dtype=np.uint8)
	for i in range(1, NUM_IMAGES + 1):
		img_file = os.path.join(training_img_directory, '{:04d}.jpg'.format(i))
		fixation_file = os.path.join(training_fixation_directory, '{:04d}.jpg'.format(i))
		train_imgs[i-1] = imageio.imread(img_file)
		fixation = imageio.imread(fixation_file)
		train_fixations[i-1] = np.expand_dims(fixation, -1) # adds singleton dimension so fixation size is (180,320,1)
	
	return train_imgs, train_fixations

# Generator function will output one (image, target) tuple at a time,
# and shuffle the data for each new epoch
def data_generator(imgs, targets):
	while True: # produce new epochs forever
		# Shuffle the data for this epoch
		idx = np.arange(imgs.shape[0])
		np.random.shuffle(idx)

		imgs = imgs[idx]
		targets = targets[idx]
		for i in range(imgs.shape[0]):
			yield imgs[i], targets[i]

def get_batch_from_generator(gen, batchsize):
	batch_imgs = []
	batch_fixations = []
	for i in range(batchsize):
		img, target = gen.__next__()
		batch_imgs.append(img)
		batch_fixations.append(target)
	return np.array(batch_imgs), np.array(batch_fixations)

In [0]:
tf.reset_default_graph()

'''
Model class
'''

class saliencyModel:
  def __init__(self, model_weights=None, learning_rate=1e-4, batch_size=32, num_batches=100, prior_downsampling_factor=10):
    self.dir_path = 'drive/My Drive/Colab Notebooks/CV2 exercies/'
    self.model_path = self.dir_path+'model_cptk/my-model1'
    self.lr_rate = learning_rate
    self.batch_size = batch_size
    self.num_batches = num_batches
    self.prior_downsampling_factor = prior_downsampling_factor
    self.input_h = 180
    self.input_w = 320
    self.prior_h = self.input_h /(2 * prior_downsampling_factor)
    self.prior_w = self.input_w / (2 * prior_downsampling_factor)
    self.reg_lambda = 1 / (self.prior_h * self.prior_w)
    
    if model_weights is not None:
      self.load_weights(model_weights)
    
  def load_weights(self, model_weights):
    vgg_weight_file = model_weights
    self.weights = np.load(vgg_weight_file)
  
  def setup(self, mode= 'Train'):
    self.input_images_placeholder = tf.placeholder(tf.uint8, [None, self.input_h, self.input_w, 3])
    self.target_images_placehodler = tf.placeholder(tf.uint8, [None, self.input_h, self.input_w, 1])
    
    with tf.name_scope('preprocessing') as scope:
      input_imgs = tf.image.convert_image_dtype(self.input_images_placeholder, tf.float32)
      fixations_normalized = tf.image.convert_image_dtype(self.target_images_placehodler, tf.float32)

    with tf.name_scope('conv1_1') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv1_1_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv1_1_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(input_imgs, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('conv1_2') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv1_2_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv1_2_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('pool1') as scope:
      pool = tf.layers.max_pooling2d(act, pool_size=(2,2), strides=(2,2), padding='same')

    with tf.name_scope('conv2_1') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv2_1_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv2_1_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(pool, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('conv2_2') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv2_2_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv2_2_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('pool2') as scope:
      pool2 = tf.layers.max_pooling2d(act, pool_size=(2,2), strides=(2,2), padding='same')

    with tf.name_scope('conv3_1') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv3_1_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv3_1_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('conv3_2') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv3_2_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv3_2_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('conv3_3') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv3_3_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv3_3_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('pool3') as scope:
      pool3 = tf.layers.max_pooling2d(act, pool_size=(2,2), strides=(1,1), padding='same')

    with tf.name_scope('conv4_1') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv4_1_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv4_1_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(pool3, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
      
    with tf.name_scope('conv4_2') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv4_2_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv4_2_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
    
    with tf.name_scope('conv4_3') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv4_3_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv4_3_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
      
    with tf.name_scope('pool4') as scope:
      pool4 = tf.layers.max_pooling2d(act, pool_size=(2,2), strides=(1,1), padding='same')
    
    with tf.name_scope('conv5_1') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv5_1_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv5_1_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(pool4, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
      
    with tf.name_scope('conv5_2') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv5_2_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv5_2_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
    
    with tf.name_scope('conv5_3') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv5_3_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv5_3_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
      conv5_3 = act
      
    with tf.name_scope('concat_featuremaps') as scope:
      concatenated_feature_maps = tf.concat([pool2, pool3, pool4, conv5_3], axis=3)
      print('IN_TRAINING_MODE is',mode =='Train')
      regularized_feature_maps = tf.layers.dropout(concatenated_feature_maps, rate=0.5, training= mode =='Train')

    with tf.name_scope('featuremaps_conv1') as scope:
      act_featuremaps_conv1 = tf.layers.conv2d(regularized_feature_maps, filters=64, kernel_size=(3,3), activation=tf.nn.relu, name='featuremaps_conv1', reuse=tf.AUTO_REUSE)

    with tf.name_scope('featuremaps_conv2') as scope:
      act_featuremaps_conv2 = tf.layers.conv2d(act_featuremaps_conv1, filters=1, kernel_size=(1,1), activation=tf.nn.relu, name='featuremaps_conv2', reuse=tf.AUTO_REUSE)

    with tf.name_scope('Learned_prior') as scope:
      #weight_initer = tf.random_normal_initializer(mean=0.0, stddev=0.1)
      #prior_shape = tf.ceil(tf.divide(act_featuremaps_conv2.shape[1:3], self.prior_downsampling_factor))
      prior_shape = (1, self.prior_h, self.prior_w,1)
      #init = tf.ones(prior_shape, dtype=tf.dtypes.float32, name='prior')
      #prior = tf.get_variable(name="prior", dtype=tf.float32, shape=prior_shape, init=weight_initer)
      prior = tf.Variable(tf.ones(prior_shape), name="prior", trainable=True)
      upsampled_prior = tf.image.resize_bilinear(prior, size=act_featuremaps_conv2.shape[1:3])
      saliency_mixed = tf.multiply(act_featuremaps_conv2, upsampled_prior)
      saliency_raw = tf.nn.relu(saliency_mixed)
      
    with tf.name_scope('loss') as scope:
      # normalize saliency
      max_value_per_image = tf.reduce_max(saliency_raw, axis=[1,2,3], keepdims=True)
      predicted_saliency = (saliency_raw / max_value_per_image)
      target_shape = predicted_saliency.shape[1:3]
      target_downscaled = tf.image.resize_images(fixations_normalized, target_shape)
      
      # Loss function from Cornia et al. (2016) [with higher weight for salient pixels]
      alpha = 1.01
      weight = 1.0 / (alpha - target_downscaled)
      loss = tf.losses.mean_squared_error(labels=target_downscaled, 
                        predictions=predicted_saliency, 
                        weights=weight)
      regularizer = tf.nn.l2_loss(1 - prior)
      loss += tf.reduce_mean(self.reg_lambda * regularizer)
      
    return predicted_saliency, loss
    
  def train(self, train_imgs, train_fixations):
    
    pred ,loss_op = self.setup(mode='Train')
    # Optimizer settings from Cornia et al. (2016) [except for decay]
    #optimizer = tf.train.MomentumOptimizer(learning_rate=self.lr_rate, momentum=0.9, use_nesterov=True)
    optimizer = tf.train.AdamOptimizer(learning_rate=self.lr_rate)
    minimize_op = optimizer.minimize(loss_op)
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
      #writer = tf.summary.FileWriter(logdir="./", graph=sess.graph)
      
      sess.run(tf.global_variables_initializer())
      #saver.restore(sess, self.model_path)
      gen = data_generator(train_imgs, train_fixations)
      for b in range(self.num_batches):
        batch_imgs, batch_fixations = get_batch_from_generator(gen, self.batch_size)
        predication, batch_loss,_ = sess.run([pred, loss_op, minimize_op], feed_dict={self.input_images_placeholder: batch_imgs, self.target_images_placehodler: batch_fixations})
        #print(predication.shape)
        if b % 10 == 0:
          #writer.add_summary(l_summary, global_step=b)
          print('Batch {0} done: batch loss {1}'.format(b, batch_loss))
          save_path = saver.save(sess, self.model_path, global_step=b)
      save_path = saver.save(sess, self.model_path)
      
  def test(self, test_imgs):
    pred_saliency, l = self.setup(mode='Test')
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
      #writer = tf.summary.FileWriter(logdir="./", graph=sess.graph)
      #saver.restore(sess, self.model_path)
      saliency = sess.run(pred_saliency, feed_dict={self.input_images_placeholder: test_imgs})
      for i in range(len(test_imgs)):
        print('saving images')
        upsampled_saliency = resize(saliency[i], output_shape=(180, 320, 1))
        saliency_img = sess.run(tf.image.convert_image_dtype(upsampled_saliency, tf.uint8))
        imageio.imwrite(self.dir_path+'/saliency/'+str(i)+'.jpg', upsampled_saliency)
    
    

In [0]:
train_imgs, train_fixations = load_train_data()

my_model = saliencyModel(model_weights='/tmp/Model/VGG16/vgg16-conv-weights.npz', learning_rate=1e-1, num_batches=101, batch_size=64)

my_model.train(train_imgs, train_fixations)
#my_model.train(train_imgs, train_fixations)

IN_TRAINING_MODE is True
Batch 0 done: batch loss 0.09543723613023758


In [43]:
my_model.test(train_imgs[0:20])


IN_TRAINING_MODE is False
INFO:tensorflow:Restoring parameters from drive/My Drive/Colab Notebooks/CV2 exercies/model_cptk/my-model1


INFO:tensorflow:Restoring parameters from drive/My Drive/Colab Notebooks/CV2 exercies/model_cptk/my-model1


saving images
saving images
saving images
saving images
saving images


saving images
saving images
saving images
saving images


saving images
saving images
saving images
saving images


saving images
saving images
saving images
saving images


saving images
saving images
saving images
